In [2]:
import pandas as pd
import re
import numpy as np

 # Hipótesis

Las hipótesis que os planteamos son:

¿Es Australia el sitio más peligroso y letal para estar relajada en la playa?

¿Hay diferencias entre los países en los ataques a hombres y mujeres?

¿Cuáles son las edades que más sufren ataques?

Independientemente de la edad, ¿sufren los hombres más ataques que las mujeres?

¿En qué mes ocurren más ataques?

¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?

¿Cómo han evolucionado los ataques a lo largo del tiempo?

Objetivos

Los objetivos de hoy son:

1. Haced una exploración inicial del dataframe:



In [3]:
df = pd.read_csv('attacks_pandas_7.csv',index_col = 0)

In [4]:
df.head(2)

,case_number,year,type,country,area,location,activity,name,age,time,species_,href,injury,date,month,fatal,sex
0,1800.00.00,1800.0,Unprovoked,SEYCHELLES,St. Anne,NaN,a corsair's boat was overturned,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,"FATAL, all onboard were killed by sharks",1800,NaN,Y,F
1,1797.05.28.R,1797.0,Unprovoked,NaN,NaN,NaN,Dropped overboard,child,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,FATAL,Reported May-28-1797,May,Y,NaN


In [5]:
pd.options.display.max_columns=None
df.head(2)

,case_number,year,type,country,area,location,activity,name,age,time,species_,href,injury,date,month,fatal,sex
0,1800.00.00,1800.0,Unprovoked,SEYCHELLES,St. Anne,NaN,a corsair's boat was overturned,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,"FATAL, all onboard were killed by sharks",1800,NaN,Y,F
1,1797.05.28.R,1797.0,Unprovoked,NaN,NaN,NaN,Dropped overboard,child,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,FATAL,Reported May-28-1797,May,Y,NaN


- ¿Cuál es el número de filas y columnas?

In [6]:
print(f'El número de filas es {df.shape[0]} y el número de columnas es {df.shape[1]}')


El número de filas es 1672 y el número de columnas es 17


- ¿Cuál es el porcentaje de valores nulos en cada columna? 
Presenta los resultados en un dataframe, donde las columnas sean el nombre de las variables y el porcentaje de nulos.

In [7]:
nulos = pd.DataFrame(df.isnull().sum()*100 / df.shape[0]).reset_index()
nulos.columns = ['variables', '% nulos'] # En este paso es cuando damos nombre a las columnas del df nulos
nulos

,variables,% nulos
0,case_number,0.000000
1,year,0.000000
2,type,0.000000
3,country,0.598086
4,area,2.751196
5,location,3.050239
6,activity,1.854067
7,name,0.538278
8,age,9.210526
9,time,9.569378


- ¿Qué tipos de datos hay en cada columna?

In [11]:
df.dtypes

case_number     object
year           float64
type            object
country         object
area            object
location        object
activity        object
name            object
age             object
time            object
species_        object
href            object
injury          object
date            object
month           object
fatal           object
sex             object
dtype: object

- Extraed la información general del dataframe.

In [ ]:
print(f'La información general del DataFrame es {df.info()}')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672 entries, 0 to 1671
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   case_number  1672 non-null   object 
 1   year         1672 non-null   float64
 2   type         1672 non-null   object 
 3   country      1662 non-null   object 
 4   area         1626 non-null   object 
 5   location     1621 non-null   object 
 6   activity     1641 non-null   object 
 7   name         1663 non-null   object 
 8   age          1518 non-null   object 
 9   time         1512 non-null   object 
 10  species_     1546 non-null   object 
 11  href         1672 non-null   object 
 12  injury       1664 non-null   object 
 13  date         1672 non-null   object 
 14  month        1518 non-null   object 
 15  fatal        1576 non-null   object 
 16  sex          1657 non-null   object 
dtypes: float64(1), object(16)
memory usage: 235.1+ KB
La información general del DataFrame es 

2. Cread dos dataframes nuevos, uno que incluya solo las variables categóricas y otro que incluya solo las variables numéricas.

In [ ]:
df_categoricas = df.select_dtypes(include = 'O')
df_categoricas.head(2)

,case_number,type,country,area,location,activity,name,age,time,species_,href,injury,date,month,fatal,sex
0,1800.00.00,Unprovoked,SEYCHELLES,St. Anne,NaN,a corsair's boat was overturned,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,"FATAL, all onboard were killed by sharks",1800,NaN,Y,F
1,1797.05.28.R,Unprovoked,NaN,NaN,NaN,Dropped overboard,child,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,FATAL,Reported May-28-1797,May,Y,NaN


In [ ]:
df_numericas = df.select_dtypes(include = ['float64','int64']) # ponemos el tipo de dato numérico que estamos buscando
df_numericas.head(2)

,year
0,1800.0
1,1797.0


- ¿Cuáles son los principales estádisticos de los dataframes?

In [ ]:
df_categoricas.describe().T # Mostramos los principales estadísticos de las columnas categóricas

,count,unique,top,freq
case_number,1672,1672,1800.00.00,1
type,1672,6,Unprovoked,1432
country,1662,95,USA,741
area,1626,236,Florida,346
location,1621,1310,"New Smyrna Beach, Volusia County",73
activity,1641,404,Surfing,426
name,1663,1536,male,97
age,1518,94,19,70
time,1512,248,11h00,74
species_,1546,856,White shark,76


In [ ]:
df_numericas.describe().T # Pricipales estadísticos de la variables numéricas

,count,mean,std,min,25%,50%,75%,max
year,1672.0,1832.688397,528.969901,0.0,1964.0,1997.0,2008.0,2018.0


In [ ]:
for col in df_categoricas.columns:
    print('----------------------------------------------------------------','\n')
    print(f'en la columna {col} tenemos la frecuencia de los siguientes valores únicos')
    print(df_categoricas[col].value_counts())
    print('\n','-----------------------------------------------------------','\n')

---------------------------------------------------------------- 

en la columna case_number tenemos la frecuencia de los siguientes valores únicos
1800.00.00      1
1990.09.05      1
1992.02.09      1
1992.03.08.a    1
1992.03.08.c    1
               ..
2009.05.16.b    1
2009.05.17      1
2009.05.31      1
2009.06.01      1
1868.05.13      1
Name: case_number, Length: 1672, dtype: int64

 ----------------------------------------------------------- 

---------------------------------------------------------------- 

en la columna type tenemos la frecuencia de los siguientes valores únicos
Unprovoked      1432
Provoked         115
Invalid           92
Boating           17
Sea Disaster      12
Boat               4
Name: type, dtype: int64

 ----------------------------------------------------------- 

---------------------------------------------------------------- 

en la columna country tenemos la frecuencia de los siguientes valores únicos
USA             741
AUSTRALIA       343
SOUT

- Extraed los valores únicos su frecuencia.
 BONUS presenta los resultados en un dataframe. 
 onde variable es cada una de las variables categóricas que tenemos en el dataset y n_unicos es una lista con los nombres de los valores únicos de cada variable.

In [ ]:
columna_list =list(df_categoricas.columns) # hacemos una lista de columnas para identificar las columnas categóricas
columna_list 

['case_number',
 'type',
 'country',
 'area',
 'location',
 'activity',
 'name',
 'age',
 'time',
 'species_',
 'href',
 'injury',
 'date',
 'month',
 'fatal',
 'sex']

In [ ]:
n_uniques = {'variables': [], 'n_uniques': []}

for col in df_categoricas.columns:
    n_uniques['variables'].append(col)
    n_uniques['n_uniques'].append(list(df_categoricas[col].unique())) 

df_n_uniques = pd.DataFrame(n_uniques)
df_n_uniques

,variables,n_uniques
0,case_number,"[1800.00.00, 1797.05.28.R, 1792.09.12, 1791.00..."
1,type,"[Unprovoked, Provoked, Boat, Sea Disaster, Inv..."
2,country,"[SEYCHELLES, nan, ENGLAND, AUSTRALIA, BARBADOS..."
3,area,"[St. Anne, nan, New South Wales, St Helena, Su..."
4,location,"[nan, On the Thames River, Port Jackson, Sydne..."
5,activity,"[a corsair's boat was overturned, Dropped over..."
6,name,"[nan, child, male, female, an Australian abori..."
7,age,"[nan, young, 14, 19, 20, 15, 21, 27, 36, 25, 6..."
8,time,"[nan, 22h00, 06h00, Night, 18h00, Morning, Aft..."
9,species_,"[nan, Tiger shark?, White sharks, Description ..."
